In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.image import imread

import rowan
import pandas as pd

In [ ]:
# please make sure this notebook and the folder '1k' with the images are in the same place
image = imread("1k/img_00001.jpg")
plt.imshow(image, cmap='gray')

In [ ]:
# reading the csv file and printing out all the label names
df = pd.read_csv("1k/robotLabel.csv")
print(df.columns)

In [ ]:
# ordering the values from the csv file
pos_uav = [df['y'][1], df['z'][1], df['x'][1], df[' qx'][1], df[' qy'][1], df[' qz '][1], df['qw'][1],]
print(pos_uav)

In [ ]:
# setting the camera intrinsics matrix and define function to calculate the pixel coordinates
camera_intrinsics = np.array([[-170, 0, 160],
                     [0, -170, 160],
                     [0,  0,  1],
                     ])

def get_pixel_coords(camera_intrinsics, pose_3d):
    u, w, z = camera_intrinsics @ pose_3d
    x_img = round(u/z)
    y_img = round(w/z)
    return x_img, y_img

In [ ]:
# sanity check if calculation works
plt.imshow(image, cmap='gray')
plt.scatter(*get_pixel_coords(camera_intrinsics, pos_uav[:3]))

In [ ]:
# calculating roation matrix with qx, qy, qz, qw
rotation = rowan.to_matrix(pos_uav[3:])
print(rotation)

In [ ]:
# setting the full transfromation matrix CF in camera frame
T_uav_in_camera = np.array([[*rotation[0], pos_uav[0]],
                            [*rotation[1], pos_uav[1]],
                            [*rotation[2], pos_uav[2]],
                            [0., 0., 0., 1.]])
print(T_uav_in_camera)

In [ ]:
# create a random patch
# the patch has a single color channel only, since the images from the dataset are grayscale
patch = (np.random.rand(100, 100, 1) * 255.).astype(int)
plt.imshow(patch, cmap='gray')

In [ ]:
# manually setting the transformation matrix
T_patch_in_uav = np.array([[0.001, 0., 0., -0.05],
                  [0., 0.001, 0., -0.05],
                  [0., 0., 0.001, 0.],
                  [0., 0., 0., 1.]])

print(T_patch_in_uav)

In [ ]:
# calculated values for the corner of the patch in UAV frame
print(T_patch_in_uav @ [0., 0., 0., 1.])
print(T_patch_in_uav @ [0., 100., 0., 1.])
print(T_patch_in_uav @ [100., 100., 0., 1.])
print(T_patch_in_uav @ [100., 0., 0., 1.])

In [ ]:
# sanity check -> scaling still fine?
fig = plt.figure()
ax = plt.axes(projection="3d")
ax.scatter3D(0., 0., 0., color='r')
ax.scatter3D(*((T_patch_in_uav @ [0., 0., 0., 1.])[:3]), color='b')
ax.scatter3D(*((T_patch_in_uav @ [0., 100., 0., 1.])[:3]), color='b')
ax.scatter3D(*((T_patch_in_uav @ [100., 100., 0., 1.])[:3]), color='b')
ax.scatter3D(*((T_patch_in_uav @ [100., 0., 0., 1.])[:3]), color='b')

In [ ]:
# sanity check for patch and uav in camera frame -> scaling still fine?
fig = plt.figure()
ax = plt.axes(projection="3d")
ax.scatter3D(*pos_uav[:3], color='r')
ax.scatter3D(*((T_uav_in_camera@T_patch_in_uav @ [0., 0., 0., 1.])[:3]), color='b')
ax.scatter3D(*((T_uav_in_camera@T_patch_in_uav @ [0., 100., 0., 1.])[:3]), color='b')
ax.scatter3D(*((T_uav_in_camera@T_patch_in_uav @ [100., 100., 0., 1.])[:3]), color='b')
ax.scatter3D(*((T_uav_in_camera@T_patch_in_uav @ [100., 0., 0., 1.])[:3]), color='b')

In [ ]:
# check if four corners of the patch end up at the right coordinates
plt.imshow(image, cmap='gray')
plt.scatter(*get_pixel_coords(camera_intrinsics, pos_uav[:3]), color='r')
plt.scatter(*get_pixel_coords(camera_intrinsics, (T_uav_in_camera@T_patch_in_uav @ [0., 0., 0., 1.])[:3]), color='b')
plt.scatter(*get_pixel_coords(camera_intrinsics, (T_uav_in_camera@T_patch_in_uav @ [100., 0., 0., 1.])[:3]), color='b')
plt.scatter(*get_pixel_coords(camera_intrinsics, (T_uav_in_camera@T_patch_in_uav @ [100., 100., 0., 1.])[:3]), color='b')
plt.scatter(*get_pixel_coords(camera_intrinsics, (T_uav_in_camera@T_patch_in_uav @ [0., 100., 0., 1.])[:3]), color='b')

In [ ]:
# now over to placing the whole patch
# first, get a 4xn matrix with all pixel-coordinates of the patch
# first pixel will we first column -> (0, 0, 0, 1)
# last pixel is last column .> (99, 99, 0, 1)
indy, indx = np.indices((patch.shape[0], patch.shape[1]), dtype=np.float32)
lin_homg_ind = np.array([indx.ravel(), indy.ravel(), np.zeros_like(indx).ravel(), np.ones_like(indx).ravel()])
print(lin_homg_ind)

In [ ]:
# calculate all x_img and y_img for the translated patch
u, w, z = camera_intrinsics@(T_uav_in_camera@T_patch_in_uav @ lin_homg_ind)[:3]

all_x = u/z
all_y = w/z
print(all_x)
print(all_y)

In [ ]:
# prepare both array -> round, they need to be integers and reshape both arrays for the following loop
all_x = np.round(all_x, decimals=0).astype(int)
all_y = np.round(all_y, decimals=0).astype(int)

all_x = all_x.reshape(patch.shape[0], patch.shape[1])
all_y = all_y.reshape(patch.shape[0], patch.shape[1])

In [ ]:
# now replace all pixels of the original image with the translated patch
for x in range(len(patch)):
    for y in range(len(patch[0])):
        image[all_y[x][y]][all_x[x][y]] = patch[x][y]

In [ ]:
# final result
plt.imshow(image, cmap='gray')